In [1]:
"""A very simple MNIST classifier.
See extensive documentation at
https://www.tensorflow.org/get_started/mnist/beginners
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data



from keras.layers.core import Dense, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D

from keras import backend as K


K.image_data_format() 

Using TensorFlow backend.


'channels_last'

In [2]:
# Import data
data_dir = '/tmp/tensorflow/mnist/input_data'
mnist = input_data.read_data_sets(data_dir, one_hot=True)
# Define loss and optimizer
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])
learning_rate = tf.placeholder(tf.float32)

with tf.name_scope('reshape'):
  x_image = tf.reshape(x, [-1, 28, 28, 1])


net = Conv2D(32, kernel_size=[5,5], strides=[1,1],activation='relu',
                 padding='same',
                input_shape=[28,28,1])(x_image)
net = MaxPooling2D(pool_size=[2,2])(net)
net = Conv2D(64, kernel_size=[5,5], strides=[1,1],activation='relu',
                padding='same')(net)
net = MaxPooling2D(pool_size=[2,2])(net)
net = Flatten()(net)
net = Dense(1000, activation='relu')(net)
net = Dense(10,activation='softmax')(net)

Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-labels-idx1-ubyte.gz


In [3]:
from keras.objectives import categorical_crossentropy
cross_entropy = tf.reduce_mean(categorical_crossentropy(y_, net))

In [4]:
l2_loss = tf.add_n( [tf.nn.l2_loss(w) for w in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)] )
total_loss = cross_entropy + 7e-5*l2_loss

train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

sess = tf.Session()

In [5]:
K.set_session(sess)

In [6]:
init_op = tf.global_variables_initializer()
sess.run(init_op)
# Train
for step in range(3000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  lr = 0.01
  _, loss, l2_loss_value, total_loss_value = sess.run(
               [train_step, cross_entropy, l2_loss, total_loss], 
               feed_dict={x: batch_xs, y_: batch_ys, learning_rate:lr})
  
  if (step+1) % 100 == 0:
    print('step %d, entropy loss: %f, l2_loss: %f, total loss: %f' % 
            (step+1, loss, l2_loss_value, total_loss_value))
    # Test trained model
    correct_prediction = tf.equal(tf.argmax(net, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print(sess.run(accuracy, feed_dict={x: batch_xs, y_: batch_ys}))
  if (step+1) % 1000 == 0:
    print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                    y_: mnist.test.labels}))

step 100, entropy loss: 1.721513, l2_loss: 789.985840, total loss: 1.776812
0.75
step 200, entropy loss: 0.667780, l2_loss: 792.024719, total loss: 0.723222
0.84
step 300, entropy loss: 0.371154, l2_loss: 792.924927, total loss: 0.426659
0.94
step 400, entropy loss: 0.357659, l2_loss: 793.474609, total loss: 0.413202
0.9
step 500, entropy loss: 0.328391, l2_loss: 793.862000, total loss: 0.383961
0.9
step 600, entropy loss: 0.376087, l2_loss: 794.153931, total loss: 0.431677
0.93
step 700, entropy loss: 0.134938, l2_loss: 794.399719, total loss: 0.190546
0.97
step 800, entropy loss: 0.280783, l2_loss: 794.632996, total loss: 0.336408
0.94
step 900, entropy loss: 0.194105, l2_loss: 794.795532, total loss: 0.249740
0.96
step 1000, entropy loss: 0.190797, l2_loss: 794.966309, total loss: 0.246445
0.95
0.9495


KeyboardInterrupt: 